In [1]:
import sys
import numpy as np
import cv2
import cv2.ml
from time import sleep
from datetime import datetime
import pysolar
import os
import numpy as np
from random import shuffle
from matplotlib import pyplot as plt
import matplotlib as mpl
from pandas import read_csv, read_excel, DataFrame
from skimage.feature import hessian_matrix_det as Hessian
from skimage.feature import local_binary_pattern as LBP
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
import pickle
import logging
logging.basicConfig(level=logging.INFO)

import joblib
import dask_jobqueue as jq
from dask.distributed import Client,wait
from dask import delayed
import dask.array as da 
p0 = os.path.join('../../','raw','CookIRCamET','Images','CookHY2023')
p1 = os.path.join('../../','work','CookIRCamET','Images','CookHY2023','RGB')
p2 = os.path.join('../../','work','CookIRCamET','Images','CookHY2023','Labels')
p00 = os.path.join('../../','raw','CookIRCamET','Images','CprlHY2023')
p11 = os.path.join('../../','work','CookIRCamET','Images','CprlHY2023','RGB')
p22 = os.path.join('../../','work','CookIRCamET','Images','CprlHY2023','Labels')
p3 = os.path.join('../../','work','CookIRCamET','Working')
n_components1 = 2
n_components2 = 4
n_components3 = 8

ModuleNotFoundError: No module named 'dask_jobqueue'

In [ ]:
cmap1 = mpl.colors.ListedColormap(['y', 'b'])
norm1 = mpl.colors.BoundaryNorm([0,1], cmap1.N)

cmap2 = mpl.colors.ListedColormap(['r', 'b', 'g','w'])
norm2 = mpl.colors.BoundaryNorm([0,1,2,3,4], cmap2.N)

# In[3]:
partition='mem'
num_processes = 16 
num_threads_per_processes = 2
mem = 32*num_processes*num_threads_per_processes
n_cores_per_job = num_processes*num_threads_per_processes

clust = jq.SLURMCluster(queue=partition,
                        processes=num_processes,
                        cores=n_cores_per_job,
                        memory=str(mem)+'GB',
                        tmpdir_ssh='/project/nsaru/py/CookIRCamET',
                        death_timeout=30,
                        walltime='7-00:00:00',
                        job_extra=["--output=/dev/null","--error=/dev/null"])
clust.scale(2)
cl = Client(clust)
dash_addr = '''/user/{}/proxy/{}/status'''.format(os.environ['USER'],cl.scheduler_info()['services']['dashboard'])
print('Dask Lab Extention Address (paste into the dask search box): '+dash_addr)
cl

In [ ]:
def localSD(mat, n):    
    mat=np.float32(mat)
    mu = cv2.blur(mat,(n,n))
    mdiff=mu-mat
    mat2=cv2.blur(np.float64(mdiff*mdiff),(n,n))
    sd = np.float32(cv2.sqrt(mat2))
    
    return sd

In [ ]:
f_imgs=[]
imgs=[]
n_img=0
for di,do in zip([p1,p11],[p2,p22]):
    fs=os.listdir(di)
    shuffle(fs)
    for f in fs:
        if 'bgr' in f:

            f_imgs = np.append(f_imgs,f)
            bgr = cv2.imread(os.path.join(di,f),cv2.IMREAD_UNCHANGED)
            time_place = f.split('_bgr.')[0].split('_')
            if len(time_place)==3 or 'nofix' in f:#V1

                labels = False
                f_labels = os.path.join(do,'SunShade',f.split('_bgr')[0]+'_class2.tif')
                if (os.path.exists(f_labels)):
                    labels1 = cv2.imread(f_labels,cv2.IMREAD_UNCHANGED)
                    labels = True

                f_labels = os.path.join(do,'SoilResVegSnow',f.split('_bgr')[0]+'_class4.tif')
                if (os.path.exists(f_labels)):
                    labels2 = cv2.imread(f_labels,cv2.IMREAD_UNCHANGED)
                    labels = (True & labels)
                if labels:         
                    print(f)
                    plt.imshow(bgr)
                    plt.show()
                    plt.imshow(labels1, cmap=cmap1, norm=norm1, interpolation='none')
                    plt.show()
                    plt.imshow(labels2, cmap=cmap2, norm=norm2, interpolation='none')
                    plt.show()
                    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
                    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
                    img = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
                    l,a,bb = cv2.split(lab)
                    h,s,v = cv2.split(hsv)

                    sd_l1 = localSD(l, 127)
                    sd_l2 = localSD(l, 63)
                    sd_l3 = localSD(l, 31)

                    lbp_l1 = LBP(l, 32, 4, method='ror')
                    lbp_l2 = LBP(l, 24, 3, method='ror')
                    lbp_l3 = LBP(l, 16, 2, method='ror')

                    sd_a1 = localSD(a, 127)
                    sd_a2 = localSD(a, 63)
                    sd_a3 = localSD(a, 31)

                    lbp_a1 = LBP(a, 32, 4, method='ror')
                    lbp_a2 = LBP(a, 24, 3, method='ror')
                    lbp_a3 = LBP(a, 16, 2, method='ror')

                    sd_b1 = localSD(bb, 127)
                    sd_b2 = localSD(bb, 63)
                    sd_b3 = localSD(bb, 31)

                    lbp_b1 = LBP(bb, 32, 4, method='ror')
                    lbp_b2 = LBP(bb, 24, 3, method='ror')
                    lbp_b3 = LBP(bb, 16, 2, method='ror')

                    sd_h1 = localSD(h, 127)
                    sd_h2 = localSD(h, 63)
                    sd_h3 = localSD(h, 31)

                    lbp_h1 = LBP(h, 32, 4, method='ror')
                    lbp_h2 = LBP(h, 24, 3, method='ror')
                    lbp_h3 = LBP(h, 16, 2, method='ror')

                    sd_s1 = localSD(s, 127)
                    sd_s2 = localSD(s, 63)
                    sd_s3 = localSD(s, 31)

                    lbp_s1 = LBP(s, 32, 4, method='ror')
                    lbp_s2 = LBP(s, 24, 3, method='ror')
                    lbp_s3 = LBP(s, 16, 2, method='ror')

                    sd_v1 = localSD(v, 127)
                    sd_v2 = localSD(v, 63)
                    sd_v3 = localSD(v, 31)

                    lbp_v1 = LBP(v, 32, 4, method='ror')
                    lbp_v2 = LBP(v, 24, 3, method='ror')
                    lbp_v3 = LBP(v, 16, 2, method='ror')

                    ddepth = cv2.CV_16S

                    lap_l1 = cv2.Laplacian(l,ddepth,ksize=3)
                    lap_l2 = cv2.Laplacian(l,ddepth,ksize=7)
                    lap_l3 = cv2.Laplacian(l,ddepth,ksize=15)

                    lap_a1 = cv2.Laplacian(a,ddepth,ksize=3)
                    lap_a2 = cv2.Laplacian(a,ddepth,ksize=7)
                    lap_a3 = cv2.Laplacian(a,ddepth,ksize=15)

                    lap_b1 = cv2.Laplacian(bb,ddepth,ksize=3)
                    lap_b2 = cv2.Laplacian(bb,ddepth,ksize=7)
                    lap_b3 = cv2.Laplacian(bb,ddepth,ksize=15)

                    lap_h1 = cv2.Laplacian(h,ddepth,ksize=3)
                    lap_h2 = cv2.Laplacian(h,ddepth,ksize=7)
                    lap_h3 = cv2.Laplacian(h,ddepth,ksize=15)

                    lap_s1 = cv2.Laplacian(s,ddepth,ksize=3)
                    lap_s2 = cv2.Laplacian(s,ddepth,ksize=7)
                    lap_s3 = cv2.Laplacian(s,ddepth,ksize=15)

                    lap_v1 = cv2.Laplacian(v,ddepth,ksize=3)
                    lap_v2 = cv2.Laplacian(v,ddepth,ksize=7)
                    lap_v3 = cv2.Laplacian(v,ddepth,ksize=15)

                    img_size = l.shape
                    bb = bb.ravel()
                    a = a.ravel()
                    l = l.ravel()
                    h = h.ravel()
                    s = s.ravel()
                    v = v.ravel()
                    sd_l1 = sd_l1.ravel()
                    sd_l2 = sd_l2.ravel()
                    sd_l3 = sd_l3.ravel()
                    lbp_l1 = lbp_l1.ravel()
                    lbp_l2 = lbp_l2.ravel()
                    lbp_l3 = lbp_l3.ravel()
                    lap_l1 = lap_l1.ravel()
                    lap_l2 = lap_l2.ravel()
                    lap_l3 = lap_l3.ravel()
                    sd_a1 = sd_a1.ravel()
                    sd_a2 = sd_a2.ravel()
                    sd_a3 = sd_a3.ravel()
                    lbp_a1 = lbp_a1.ravel()
                    lbp_a2 = lbp_a2.ravel()
                    lbp_a3 = lbp_a3.ravel()
                    lap_a1 = lap_a1.ravel()
                    lap_a2 = lap_a2.ravel()
                    lap_a3 = lap_a3.ravel()
                    sd_b1 = sd_b1.ravel()
                    sd_b2 = sd_b2.ravel()
                    sd_b3 = sd_b3.ravel()
                    lbp_b1 = lbp_b1.ravel()
                    lbp_b2 = lbp_b2.ravel()
                    lbp_b3 = lbp_b3.ravel()
                    lap_b1 = lap_b1.ravel()
                    lap_b2 = lap_b2.ravel()
                    lap_b3 = lap_b3.ravel()
                    sd_h1 = sd_h1.ravel()
                    sd_h2 = sd_h2.ravel()
                    sd_h3 = sd_h3.ravel()
                    lbp_h1 = lbp_h1.ravel()
                    lbp_h2 = lbp_h2.ravel()
                    lbp_h3 = lbp_h3.ravel()
                    lap_h1 = lap_h1.ravel()
                    lap_h2 = lap_h2.ravel()
                    lap_h3 = lap_h3.ravel()
                    sd_s1 = sd_s1.ravel()
                    sd_s2 = sd_s2.ravel()
                    sd_s3 = sd_s3.ravel()
                    lbp_s1 = lbp_s1.ravel()
                    lbp_s2 = lbp_s2.ravel()
                    lbp_s3 = lbp_s3.ravel()
                    lap_s1 = lap_s1.ravel()
                    lap_s2 = lap_s2.ravel()
                    lap_s3 = lap_s3.ravel()
                    sd_v1 = sd_v1.ravel()
                    sd_v2 = sd_v2.ravel()
                    sd_v3 = sd_v3.ravel()
                    lbp_v1 = lbp_v1.ravel()
                    lbp_v2 = lbp_v2.ravel()
                    lbp_v3 = lbp_v3.ravel()
                    lap_v1 = lap_v1.ravel()
                    lap_v2 = lap_v2.ravel()
                    lap_v3 = lap_v3.ravel()
                    feat = np.vstack((l.T,a.T,bb.T,h.T,s.T,v.T,
                                      sd_l1.T,sd_l2.T,sd_l3.T,
                                      lbp_l1.T,lbp_l2.T,lbp_l3.T,
                                      lap_l1.T,lap_l2.T,lap_l3.T,
                                      sd_a1.T,sd_a2.T,sd_a3.T,
                                      lbp_a1.T,lbp_a2.T,lbp_a3.T,
                                      lap_a1.T,lap_a2.T,lap_a3.T,
                                      sd_b1.T,sd_b2.T,sd_b3.T,
                                      lbp_b1.T,lbp_b2.T,lbp_b3.T,
                                      lap_b1.T,lap_b2.T,lap_b3.T,
                                      sd_h1.T,sd_h2.T,sd_h3.T,
                                      lbp_h1.T,lbp_h2.T,lbp_h3.T,
                                      lap_h1.T,lap_h2.T,lap_h3.T,
                                      sd_s1.T,sd_s2.T,sd_s3.T,
                                      lbp_s1.T,lbp_s2.T,lbp_s3.T,
                                      lap_s1.T,lap_s2.T,lap_s3.T,
                                      sd_v1.T,sd_v2.T,sd_v3.T,
                                      lbp_v1.T,lbp_v2.T,lbp_v3.T,
                                      lap_v1.T,lap_v2.T,lap_v3.T)).T
                    #labels = np.sum(np.vstack((soil.ravel().T, residue.ravel().T*2, shadow.ravel().T*3, vegetation.ravel().T*4)).T,axis=1)
                    labels1 = labels1.ravel()        
                    labels2 = labels2.ravel() 
                    #8-class
                    labels3 = 4*labels1+labels2   

                    if not np.any(np.isnan(feat)):
                        imgs.append({'bgr':bgr,'feats':feat,'labels1':labels1,'labels2':labels2,'labels3':labels3})
                        n_img=n_img+1
n_feat = feat.shape[1]

In [5]:
feats = []
labels1 = []
labels2 = []
labels3 = []
for sample in imgs:
    feats.append(sample['feats'])
    labels1.append(sample['labels1'])
    labels2.append(sample['labels2'])
    labels3.append(sample['labels3'])

In [6]:
feats = np.array(feats).reshape((-1,n_feat)).astype(np.float32)
labels1 = np.array(labels1).reshape((-1,1)).astype(np.int32).ravel()
labels2 = np.array(labels2).reshape((-1,1)).astype(np.int32).ravel()
labels3 = np.array(labels3).reshape((-1,1)).astype(np.int32).ravel()
scaler = StandardScaler()
filename = os.path.join(p3,'scaler.pk.sav')
feats = scaler.fit_transform(feats)
pickle.dump(scaler, open(filename, 'wb'))

In [7]:
train_feats3, test_feats3, train_labels3, test_labels3 = train_test_split(feats, labels3, test_size=0.2, random_state=42)

In [8]:
def cornfusion(obs,pred,nclass):
    M = np.zeros((nclass,nclass))
    for i in range(obs.shape[0]):
        o = obs[i]
        p = pred[i]
        M[o,p] = M[o,p]+1
    return M

In [9]:
train_feats = train_feats3#[:,mask3]
test_feats = test_feats3#[:,mask3]

In [ ]:
parameters = {'max_leaf_nodes':(10,31,100),
              'min_samples_leaf':(10,20,40),
              'max_bins':(127,255,511)}
hgb = HistGradientBoostingClassifier(max_iter=100000,random_state=42)
clf_hgb3 = GridSearchCV(hgb, parameters,n_jobs=-1,cv=5)
clf_hgb3.fit(train_feats, train_labels3)

model_hgb3 = clf_hgb3.best_estimator_
pred_hgb3 = clf_hgb3.predict(test_feats)

In [ ]:
M_hgb3 = cornfusion(test_labels3,pred_hgb3,n_components3)
M_hgb3 = M_hgb3/np.sum(np.sum(M_hgb3))
recall3 = np.diag(M_hgb3)/np.sum(M_hgb3,axis=1)
precis3 = np.diag(M_hgb3)/np.sum(M_hgb3,axis=0)


M_hgb4 = cornfusion(test_labels3,4*pred_hgb1+pred_hgb2,n_components3)
M_hgb4 = M_hgb4/np.sum(np.sum(M_hgb4))
recall4 = np.diag(M_hgb4)/np.sum(M_hgb4,axis=1)
precis4 = np.diag(M_hgb4)/np.sum(M_hgb4,axis=0)
M_hgb4

f3=(recall3*precis3/(recall3+precis3)*2)
f3_weighted=np.sum(f3*np.sum(M_hgb3, axis=1))

f4=(recall4*precis4/(recall4+precis4)*2)
f4_weighted=np.sum(f4*np.sum(M_hgb4, axis=1))
print(f3)
print(f3_weighted)
print(f4)
print(f4_weighted)

In [ ]:
filename = os.path.join(p3,'finalized_model3_hgb.pk.sav')
pickle.dump(model_hgb3, open(filename, 'wb'))

In [ ]:
M_hgb1_df = {}
M_hgb1_df['sun'] = M_hgb1[:,0]
M_hgb1_df['shade'] = M_hgb1[:,1]
M_hgb1_df = DataFrame(M_hgb1_df)
M_hgb1_df.to_csv(os.path.join(p3,'M1_hgb.csv'))

M_hgb2_df = {}
M_hgb2_df['soil'] = M_hgb2[:,0]
M_hgb2_df['res'] = M_hgb2[:,1]
M_hgb2_df['can'] = M_hgb2[:,2]
M_hgb2_df['snow'] = M_hgb2[:,3]
M_hgb2_df = DataFrame(M_hgb2_df)
M_hgb2_df.to_csv(os.path.join(p3,'M2_hgb.csv'))

M_hgb3_df = {}
M_hgb3_df['sun_soil'] = M_hgb3[:,0]
M_hgb3_df['sun_res'] = M_hgb3[:,1]
M_hgb3_df['sun_can'] = M_hgb3[:,2]
M_hgb3_df['sun_snow'] = M_hgb3[:,3]
M_hgb3_df['shade_soil'] = M_hgb3[:,4]
M_hgb3_df['shade_res'] = M_hgb3[:,5]
M_hgb3_df['shade_can'] = M_hgb3[:,6]
M_hgb3_df['shade_snow'] = M_hgb3[:,7]
M_hgb3_df = DataFrame(M_hgb3_df)
M_hgb3_df.to_csv(os.path.join(p3,'M3_hgb.csv'))

M_hgb4_df = {}
M_hgb4_df['sun_soil'] = M_hgb4[:,0]
M_hgb4_df['sun_res'] = M_hgb4[:,1]
M_hgb4_df['sun_can'] = M_hgb4[:,2]
M_hgb4_df['sun_snow'] = M_hgb4[:,3]
M_hgb4_df['shade_soil'] = M_hgb4[:,4]
M_hgb4_df['shade_res'] = M_hgb4[:,5]
M_hgb4_df['shade_can'] = M_hgb4[:,6]
M_hgb4_df['shade_snow'] = M_hgb4[:,7]
M_hgb4_df = DataFrame(M_hgb4_df)
M_hgb4_df.to_csv(os.path.join(p3,'M4_hgb.csv'))

p1_df = DataFrame(clf_hgb1.best_params_)
p1_df.to_csv(os.path.join(p3,'params1_hgb.csv'))

p2_df = DataFrame(clf_hgb2.best_params_)
p2_df.to_csv(os.path.join(p3,'params2_hgb.csv'))

p3_df = DataFrame(clf_hgb3.best_params_)
p3_df.to_csv(os.path.join(p3,'params3_hgb.csv'))